In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from smallervggnet import SmallerVGGNet
from imutils import paths
import glob
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (100, 100, 3)

data = []
labels = []
Abs_path='content/gdrive/My Drive/Colab Notebooks/CNN/train/'
print("[INFO] loading images...")
for imgPath in list(paths.list_images(Abs_path)):
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imgPath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    label=imgPath.split('\\')[-2]
    labels.append(label)

In [0]:
from sklearn.preprocessing import LabelBinarizer
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
(trainX, testX, trainY, testY) = train_test_split(data,labels,
test_size=0.2, random_state=42)

In [0]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
horizontal_flip=True, fill_mode="nearest")

In [0]:
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

In [0]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(
aug.flow(trainX, trainY, batch_size=BS),
validation_data=(testX, testY),
steps_per_epoch=len(trainX) // BS,
epochs=EPOCHS, verbose=1)

In [0]:
# saving the model
print("[INFO] serializing network...")
model.save('Plant_classifier.model')

print("[INFO] serializing label binarizer...")
f = open('labels.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

In [0]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(args["plot"])